In [17]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pickle
import pandas as pd
from sklearn.metrics import accuracy_score
import random

In [2]:
def calcul_L0(language):
    return set(language)

def calcul_L1(language):
    resid = quotient(language,language)
    resid.remove("")
    return resid


def residuel(mot, language):
    residuel_set = set()
    for word in language:
        if word.startswith(mot):
            residuel_set.add(word[len(mot):])
    return residuel_set

def setMoi(set1, set2):
    for element in set2:
        set1.add(element)

def quotient(language1, language2):
    quotient_set = set()
    for word in language1:
        residual_set = residuel(word, language2)
        setMoi(quotient_set,residual_set)
    return quotient_set

def sardinas(language):
    l0 = calcul_L0(language)
    if(len(l0) == 1):
        return True
    else:
        l1 = calcul_L1(l0)
        tab_set = [l0,l1]
        return ln(tab_set,2)

def ln(tab_set,indice):
    lang_princ = tab_set[0]
    lang_moins = tab_set[indice-1]
    valiny = ln_mitohy(lang_princ,lang_moins)
    tab_set.append(valiny)

    if len(valiny) == 0:
        return True
    elif contient_eps(valiny) == True:
        return False
    elif has_duplicates(tab_set) == True:
        return True
    else:
        return ln(tab_set,indice+1)


def ln_mitohy(lang_principal,lang_moins):
    resid1 = quotient(lang_principal,lang_moins)
    resid2 = quotient(lang_moins,lang_principal)
    setMoi(resid1,resid2)
    return resid1

def contient_eps(set1):
    for item in set1:
        if item == "":
            return True
    return False

def has_duplicates(tab):
    tuple_set = set()
    for s in tab:
        tuple_set.add(tuple(sorted(s)))
    
    if len(tuple_set) < len(tab):
        return True
    else:
        return False


In [3]:
def longueur_moyenne(language):
    taille = len(language)
    somme = 0
    for el in language:
        somme += len(el)
    return somme/taille

def about_1_0(language):
    taille = len(language)
    taille_total = 0
    nb_0 = 0
    nb_1 = 0
    for el in language:
        taille_total += len(el)
        nb_0 += el.count('0')
        nb_1 += el.count('1')
    moyenne_nb_0 = nb_0/taille
    moyenne_nb_1 = nb_1/taille
    p_nb_0 = nb_0/taille_total
    p_nb_1 = nb_1/taille_total

    return moyenne_nb_0,moyenne_nb_1,p_nb_0,p_nb_1

In [135]:
def language_to_matrix(language):
    lgm = longueur_moyenne(language)
    moyenne_nb_0,moyenne_nb_1,p_nb_0,p_nb_1 = about_1_0(language)
    nb_mot = len(language)
    return [nb_mot,lgm,moyenne_nb_0,moyenne_nb_1,p_nb_0,p_nb_1]

def language_to_matrix2(language,all_binary_words):
    matr = language_to_matrix(language)
    for words in all_binary_words:
        if words in language:
            matr.append(1)
        else:
            matr.append(0)
    return matr

def donneeToMatrix(donnee,all_binary_words = []):
    tab_valeur = []
    tab_matrix = []
    for d in donnee:
        tab_matrix.append(language_to_matrix2(d,all_binary_words))
        tab_valeur.append(sardinas(d))
    return tab_valeur, tab_matrix

In [5]:
def apprentissage(valeur,matrix):
    X = np.array(matrix)
    y = np.array(valeur)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25,random_state = 1)
    clf = RandomForestClassifier(random_state=0)
    clf = clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    a = accuracy_score(y_test, y_pred)
    print(a)
    return clf

def prediction(clf,matrix):
    valiny =  clf.predict([matrix])
    return valiny[0]

def ecrireFichier(clf):
        variable_serializee = pickle.dumps(clf)
        with open("modele.pkl", "wb") as fichier:
            fichier.write(variable_serializee)

def lireFichier():
        with open("modele.pkl", "rb") as fichier:
            contenu_fichier = fichier.read()
        return pickle.loads(contenu_fichier)

In [29]:
import itertools

def generate_all_binary_words(min_length, max_length):
    all_words = []
    for length in range(min_length, max_length + 1):
        all_words.extend([''.join(bits) for bits in itertools.product('01', repeat=length)])
    return all_words

# Générer toutes les combinaisons possibles de mots binaires de longueur 1 à 7
min_length = 1
max_length = 7
all_binary_words = generate_all_binary_words(min_length, max_length)


In [54]:
def generate_all_languages(words, min_words=2, max_words=10):
    all_languages = set()
    
    for num_words in range(min_words, max_words + 1):
        for language in itertools.combinations(words, num_words):
            all_languages.add(tuple(sorted(language)))
        print(num_words)
    
    return all_languages
        
def printtab(tab):
    for a in tab:
        print(a)

In [110]:
def genererMyLanguage(all_binary_words):
    nombre = 3000
    li = set()
    tab_augment = []

    for a in all_binary_words:
        tab_augment.append(set([a]))

    valiny = ge_more1(tab_augment,all_binary_words)
    li.update(random.sample(valiny,nombre))
    tab_augment = random.sample(valiny,nombre)
    tab_augment = nettoyer(tab_augment)
    for i in range (3,11):
        val = ge_moreAll(tab_augment,all_binary_words, i)
        li.update(random.sample(val,nombre))
        tab_augment = random.sample(val,nombre)
        tab_augment = nettoyer(tab_augment)
    return li

def nettoyer(tabtuple):
    ma = []
    for a in tabtuple:
        ma.append(set(a))
    return ma

def ge_more1(tab_augment,all_binary_words):
    valiny = set()
    for a in tab_augment:
        for el in all_binary_words:
            new_tab = set([el]) | a
            if(len(new_tab) == 2):
                salut = tuple(sorted(new_tab))
                valiny.add(salut)
    return valiny

def ge_moreAll(tab_augment,all_binary_words,indice):
    valiny = set()
    for a in tab_augment:
        for el in all_binary_words:
            new_tab = set([el]) | a
            if(len(new_tab) == indice):
                salut = tuple(sorted(new_tab))
                valiny.add(salut)
    return valiny

In [136]:
mylanguages= genererMyLanguage(all_binary_words)

/tmp/ipykernel_9499/3096975828.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  li.update(random.sample(valiny,nombre))
/tmp/ipykernel_9499/3096975828.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  tab_augment = random.sample(valiny,nombre)
/tmp/ipykernel_9499/3096975828.py:15: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  li.update(random.sample(val,nombre))
/tmp/ipykernel_9499/3096975828.py:16: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  tab_augment = random.sample(val,nombre)


In [137]:
# print(len(mylanguages))
code  = []
non_code  = []
tab = {True:0,False:0}
for lan in mylanguages:
    sard = sardinas(lan)
    if sard:
        code.append(lan)
    else:
        non_code.append(lan)

code = random.sample(code,5000)
print(len(code), len(non_code))

5000 3399


In [139]:
import csv
def ecrire(fie,language):
    with open(fie,'w',newline='') as file:
        writer  = csv.writer(file,delimiter=';')
        writer.writerows(language)

ecrire('mycode.csv',code)

In [67]:
def factorial(n):
    if n == 0 or n == 1:
        return 1
    result = 1
    for i in range(2, n + 1):
        result *= i
    return result

def combinations(n, p):
    return factorial(n) // (factorial(p) * factorial(n - p))

somme = 0
for i in range (2,11):
    comb = combinations(254,i)
    print(i, " : ", comb)
    somme += comb
# print("total: ",somme)
# print("comb", combinations(9,2))


2  :  32131
3  :  2699004
4  :  169362501
5  :  8468125050
6  :  351427189575
7  :  12450563287800
8  :  384411141510825
9  :  10507237867962550
10  :  257427327765082475


In [131]:
df = pd.read_csv('mycode.csv')
df_n = pd.read_csv('mytsy_code.csv')

In [132]:
X = df.iloc[:, :].values
X_n = df_n.iloc[:, :].values
languages = []
for d in X:
    languages.append(d[0].split(';'))

for d in X_n:
    languages.append(d[0].split(';'))


In [133]:
tab_valeur, tab_matrix = donneeToMatrix(languages,all_binary_words)
clf = apprentissage(tab_valeur,tab_matrix)
ecrireFichier(clf)

0.8213592233009709


In [129]:
language = ["000", "01","100", "010"]
mat_lang = language_to_matrix2(language,all_binary_words)
clf = lireFichier()
val = prediction(clf,mat_lang)
sard = sardinas(language)
print(val, sard)

True True
